In [ ]:
pip install torch torchvision
pip install matplotlib
pip install streamlit

     -------------------------------------- 216.1/216.1 MB 4.2 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 6.8 MB/s eta 0:00:00
     -------------------------------------- 199.1/199.1 KB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 5.8 MB/s eta 0:00:00
     ---------------------------------------- 6.3/6.3 MB 5.8 MB/s eta 0:00:00
     -------------------------------------- 134.9/134.9 KB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 12.9/12.9 MB 6.0 MB/s eta 0:00:00
     ---------------------------------------- 2.7/2.7 MB 5.9 MB/s eta 0:00:00
     -------------------------------------- 536.2/536.2 KB 4.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\ASUS\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
pip install torch

In [1]:
import os, json
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.nn as nn, torch.optim as optim

In [ ]:
# Setup dataset
data_dir = "dataset/"  # ganti sesuai lokasi

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])
dataset = datasets.ImageFolder(data_dir, transform=transform)
train_size = int(0.8*len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = torch.utils.data.random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)

# Save mapping
class_mapping = {v:k for k,v in dataset.class_to_idx.items()}
with open("class_mapping.json", "w") as f:
    json.dump(class_mapping, f)

# Build model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(dataset.classes))
model = model.to(device)

# Training
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

for epoch in range(5):
    model.train()
    total_loss = 0
    correct = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        out = model(imgs)
        loss = criterion(out, labels)
        loss.backward(); optimizer.step()
        total_loss += loss.item()
        correct += (out.argmax(1)==labels).sum().item()
    acc = correct/len(train_ds)
    print(f"Epoch {epoch+1}: loss {total_loss/len(train_loader):.4f}, acc {acc:.4f}")

# Save
torch.save(model.cpu().state_dict(), "fish_freshness_model.pth")
print("Selesai training ✅")


c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1: loss 1.7009, acc 0.4744
Epoch 2: loss 0.5967, acc 0.8249
Epoch 3: loss 0.2467, acc 0.9556
Epoch 4: loss 0.0885, acc 0.9935
Epoch 5: loss 0.0398, acc 0.9991
Selesai training ✅


In [3]:
# Evaluasi akurasi di data validasi
model.eval()
correct = 0
with torch.no_grad():
    for imgs, labels in val_loader:
        out = model(imgs)
        pred = out.argmax(1)
        correct += (pred == labels).sum().item()

val_acc = correct / len(val_ds)
print(f"Akurasi di data validasi: {val_acc:.4f}")

Akurasi di data validasi: 0.7711
